In [17]:
#
# TODO: Agregar data básica del modelo
# * Agregar URLs a publicaciones, imágenes
# TODO: Hacer segundo research en base a los modelos a comparar
# * Los modelos del comparador algunos están descontinuados, se deberían de quitar
# ! Los modelos de la base del comparador no están completos, trae unos pocos
# ? SOP: https://docs.google.com/document/d/1zp91sPv0uYKKByQpszNrFQEFBcFzxP0HXX5vgh1azVw/edit?usp=sharing

In [18]:
from google import genai
import os
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from config.paths import SRC_DIR
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [19]:
pais = "Colombia"

## Generar los datos básicos del modelo

### DB del comparador

In [28]:
df_comparador = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\src\data\db\bd_comparador.csv")
df_comparador = df_comparador[df_comparador["stock"] == "disponible"]

,url,Views,stock,stock_detail
2,https://www.galgo.com/co/comparador/motos/CO728-honda-xr150l-vs-CO1757-suzuki-dr-150-fi-abs-2025,2543,disponible,"{""CO728-honda-xr150l"": ""disponible"", ""CO1757-suzuki-dr-150-fi-abs-2025"": ""disponible""}"
4,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO731-tvs-raider-125,2291,disponible,"{""CO2961-hero-hunk-125-r"": ""disponible"", ""CO731-tvs-raider-125"": ""disponible""}"
5,https://www.galgo.com/co/comparador/motos/CO1266-suzuki-gixxer-sf-150-abs-vs-CO1262-suzuki-gsx-r150-abs,2243,disponible,"{""CO1266-suzuki-gixxer-sf-150-abs"": ""disponible"", ""CO1262-suzuki-gsx-r150-abs"": ""disponible""}"
13,https://www.galgo.com/co/comparador/motos/CO928-akt-motos-nkd-125-vs-CO731-tvs-raider-125,1377,disponible,"{""CO928-akt-motos-nkd-125"": ""disponible"", ""CO731-tvs-raider-125"": ""disponible""}"
14,https://www.galgo.com/co/comparador/motos/CO930-akt-motos-tt-dual-sport-200-vs-CO942-akt-motos-ttr-200,1295,disponible,"{""CO930-akt-motos-tt-dual-sport-200"": ""disponible"", ""CO942-akt-motos-ttr-200"": ""disponible""}"
...,...,...,...,...
936,https://www.galgo.com/co/comparador/motos/CO914-kawasaki-z1000-r-vs-CO1371-yamaha-mt10,126,disponible,"{""CO914-kawasaki-z1000-r"": ""disponible"", ""CO1371-yamaha-mt10"": ""disponible""}"
975,https://www.galgo.com/co/comparador/motos/CO1384-yamaha-mt09-vs-CO1367-yamaha-r15,120,disponible,"{""CO1384-yamaha-mt09"": ""disponible"", ""CO1367-yamaha-r15"": ""disponible""}"
984,https://www.galgo.com/co/comparador/motos/CO928-akt-motos-nkd-125-vs-CO594-victory-mrx-150,119,disponible,"{""CO928-akt-motos-nkd-125"": ""disponible"", ""CO594-victory-mrx-150"": ""disponible""}"
995,https://www.galgo.com/co/comparador/motos/CO729-honda-xr-190l-vs-CO2772-suzuki-vstrom-160,112,disponible,"{""CO729-honda-xr-190l"": ""disponible"", ""CO2772-suzuki-vstrom-160"": ""disponible""}"


### Base de inventario

In [22]:
co_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\CO\BaseCOv2.csv"
mx_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\MX\BaseMXv5.csv"
cl_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\CL-MOTORCYCLES\df_cl-motorcycles_history.csv"

In [23]:
if pais == "Colombia":
    base_data = pd.read_csv(co_inventory)
    country_code = "CO"
elif pais == "Mexico":
    base_data = pd.read_csv(mx_inventory)
    country_code = "MX"
elif pais == "Chile":
    base_data = pd.read_csv(cl_inventory)
    country_code = "CL"

C:\Users\JTRUJILLO\AppData\Local\Temp\ipykernel_12836\1272235273.py:2: DtypeWarning: Columns (8,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  base_data = pd.read_csv(co_inventory)


In [24]:
# Filtrar por fecha de actualización
filtro_fecha = base_data["Fecha de actualización"] == "22/12/2025"
base_filtrada = base_data[filtro_fecha]

# Filtrar por estado disponible o sin stock
estados_validos = ["available", "no_stock"]
base_filtrada = base_filtrada[base_filtrada["Estado"].isin(estados_validos)]

# Excluir los que tienen redirección
base_filtrada = base_filtrada[base_filtrada["tiene_redirección"] == False]

# Seleccionar solo las columnas necesarias
columnas_a_usar = ["Fecha de actualización", "code", "Marca", "Modelo", "Año", "Precio Neto", "Tipo"]
df_columnas_seleccionadas = base_filtrada[columnas_a_usar].copy()
df_columnas_seleccionadas = df_columnas_seleccionadas.rename(columns={"Precio Neto": "Precio"})

# Eliminar duplicados por código
df_desduplicadas = df_columnas_seleccionadas.drop_duplicates(subset="code").copy()

# Agregar columna de URL
df_desduplicadas["url"] = df_desduplicadas["code"].apply(
    lambda x: f"https://www.galgo.com/{country_code.lower()}/motos/{x}"
)

# Definir columnas finales
columnas_finales = ["Fecha de actualización", "code", "Marca", "Modelo", "Año", "Tipo", "Precio", "url"]
df_columnas_finales = df_desduplicadas[columnas_finales]

In [25]:
df_modelo = df_columnas_finales[df_columnas_finales["code"] == "CO2961-hero-hunk-125-r"]

In [26]:
for index, row in df_modelo.iterrows():
    marca = row["Marca"]
    modelo = row["Modelo"]
    año = row["Año"]
    pais = pais
    tipo = row["Tipo"]
    code = row["code"]

In [27]:
df_comparador[df_comparador["url"].str.contains(code)].head(3)

,url,Views,stock,stock_detail
4,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO731-tvs-raider-125,2291,disponible,"{""CO2961-hero-hunk-125-r"": ""disponible"", ""CO731-tvs-raider-125"": ""disponible""}"
19,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO2861-tvs-raider-fi-125,1272,disponible,"{""CO2961-hero-hunk-125-r"": ""disponible"", ""CO2861-tvs-raider-fi-125"": ""disponible""}"
41,https://www.galgo.com/co/comparador/motos/CO1041-bajaj-pulsar-ns-125-vs-CO2961-hero-hunk-125-r,874,no_disponible,"{""CO1041-bajaj-pulsar-ns-125"": ""no_disponible"", ""CO2961-hero-hunk-125-r"": ""disponible""}"


### Prompt a enviar

In [ ]:
from utils.prompt_utils import *

prompt_from_file = read_prompt_from_file(os.path.join(SRC_DIR, "data", "prompts", "knowledge_base_template.md"))
prompt = replace_variables(prompt_from_file, {
    "{marca}": marca,
    "{modelo}": modelo,
    "{año}": str(año),
    "{pais}": pais,
    "{tipo}": tipo
})

### Usando Gemini

In [ ]:
client = genai.Client() # La API Key se lee automaticamente desde variables de entorno

# Llamada a la API
interaction_con = client.interactions.create(
    model='gemini-3-pro-preview',
    input=prompt,
    tools=[{'type': 'google_search'}]  # Habilita Google Search
)

### Output de la API

In [ ]:
final_output = ""
for output in interaction_con.outputs:
    if hasattr(output, "text"):
        final_output += output.text

### Exportar OUTPUT

In [ ]:
with open(f"{SRC_DIR}/data/output_gemini/{code}-{pais}.md", "w", encoding="utf-8") as f:
    f.write(final_output)

### Usando Replicate

In [ ]:
# client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

# output = client.run(
#     ref = "google/gemini-3-pro",
#     input={
#         "prompt": prompt
#     },
# )

# print("\n--- RESPUESTA ---\n")
# if isinstance(output, list):
#     print("".join([str(x) for x in output]))
# else:
#     print(output)